# YOLOv5로 돼지 탐지하기

### 환경
-   KiSTi 국가컴퓨팅센터 - 뉴론(NEURON)

-   운영체제: CentOS 7.9 (Linux, 64-bit)

-   CPU: Intel Xeon Ivy Bridge (E5-2670) / 2.50GHz (10-core) / 2 socket

-   GPU: Tesla V100-PCIE-16GB(2개중 하나만 사용함)

-   메인 메모리: 128GB DDR3 Memory

-   CUDA Version: 11.6

-   Python Version: 3.9.13

In [ ]:
# 깃허브에서 YOLOv5 오픈소스 가져오기
!git clone https://github.com/ultralytics/yolov5

In [ ]:
# 필요한 패키지 한번에 설치해주기
%cd yolov5
!pip install -qr requirements.txt

In [3]:
from typing_extensions import IntVar
import json
import os
import random


def split_dataset(input_json, output_dir, val_ratio, test_ratio, random_seed):
    random.seed(random_seed)

    # 원본 json파일을 불러오기
    with open(input_json) as json_reader:
        dataset = json.load(json_reader)

    images = dataset['images']
    annotations = dataset['annotations']
    categories = dataset['categories']

    # 이미지별 id가져오기
    image_ids = [x.get('id') for x in images]
    image_ids.sort()
    # 무작위로 섞기
    random.shuffle(image_ids)
    
    # train/validation/test로 나누어 주기
    num_val = int(len(image_ids) * val_ratio)
    num_test = int(len(image_ids) * test_ratio)
    num_train = int(len(image_ids)) - num_val - num_test

    image_ids_val, image_ids_train , image_ids_test= set(image_ids[:num_val]), set(image_ids[num_val:num_train]), set(image_ids[num_train:])

    # 이미지 id와 annotations도 위에서 나눈대로 분류해주기
    train_images = [x for x in images if x.get('id') in image_ids_train]
    val_images = [x for x in images if x.get('id') in image_ids_val]
    test_images = [x for x in images if x.get('id') in image_ids_test]
    train_annotations = [x for x in annotations if x.get('image_id') in image_ids_train]
    val_annotations = [x for x in annotations if x.get('image_id') in image_ids_val]
    test_annotations = [x for x in annotations if x.get('image_id') in image_ids_test]

    train_data = {
        'images': train_images,
        'annotations': train_annotations,
        'categories': categories,
    }

    val_data = {
        'images': val_images,
        'annotations': val_annotations,
        'categories': categories,
    }

    test_data = {
        'images': test_images,
        'annotations': test_annotations,
        'categories': categories,
    }

    # 위에서 분류한 것들을 세개의 json파일로 나누어서 저장해주기
    output_seed_dir = os.path.join(output_dir, f'seed{random_seed}')
    os.makedirs(output_seed_dir, exist_ok=True)
    output_train_json = os.path.join(output_seed_dir, 'train.json')
    output_val_json = os.path.join(output_seed_dir, 'val.json')
    output_test_json = os.path.join(output_seed_dir, 'test.json')
    output_train_csv = os.path.join(output_seed_dir, 'train.csv')
    output_val_csv = os.path.join(output_seed_dir, 'val.csv')
    output_test_csv = os.path.join(output_seed_dir, 'test.csv')

    print(f'write {output_train_json}')
    with open(output_train_json, 'w') as train_writer:
        json.dump(train_data, train_writer)

    print(f'write {output_val_json}')
    with open(output_val_json, 'w') as val_writer:
        json.dump(val_data, val_writer)

    print(f'write {output_test_json}')
    with open(output_test_json, 'w') as test_writer:
        json.dump(test_data, test_writer)

In [ ]:
# 하나로 모여있는 2700장 데이터를 train/validation/test로 나누어준다(비율은 70%.20%/10%이다)
split_dataset(input_json='/scratch/a1589a01/workspace/YOLO/data/Train.vol1/json_1/annotation.json',
              output_dir='/scratch/a1589a01/workspace/YOLO/data/Train.vol1/json_1',
              val_ratio=0.2,
              test_ratio=0.1,
              random_seed=221111)

In [ ]:
# COCO 데이터셋을 YOLO 데이터셋으로 변경해주는 오픈소스를 깃허브에서 가져오기
!git clone https://github.com/alexmihalyk23/COCO2YOLO.git

In [5]:
import shutil

class COCO2YOLO:
  # 소스 이미지 디렉토리와 Json annotation 파일, 타겟 이미지 디렉토리, 타겟 annotation 디렉토리를 생성자로 입력 받음
  def __init__(self, src_img_dir, json_file, tgt_img_dir, tgt_anno_dir):
    self.json_file = json_file
    self.src_img_dir = src_img_dir
    self.tgt_img_dir = tgt_img_dir
    self.tgt_anno_dir = tgt_anno_dir
    # json 파일과 타겟 디렉토리가 존재하는지 확인하고, 디렉토리의 경우는 없으면 생성
    self._check_file_and_dir(json_file, tgt_img_dir, tgt_anno_dir)
    # json 파일을 메모리로 로딩. 
    self.labels = json.load(open(json_file, 'r', encoding='utf-8'))
    # category id와 이름을 매핑하지만, 실제 class id는 이를 적용하지 않고 별도 적용
    self.coco_id_name_map = self._categories()
    self.coco_name_list = list(self.coco_id_name_map.values())
    print("total images", len(self.labels['images']))
    print("total categories", len(self.labels['categories']))
    print("total labels", len(self.labels['annotations']))
  
  # json 파일과 타겟 디렉토리가 존재하는지 확인하고, 디렉토리의 경우는 없으면 생성
  def _check_file_and_dir(self, file_path, tgt_img_dir, tgt_anno_dir):
    if not os.path.exists(file_path):
        raise ValueError("file not found")
    if not os.path.exists(tgt_img_dir):
        os.makedirs(tgt_img_dir)
    if not os.path.exists(tgt_anno_dir):
        os.makedirs(tgt_anno_dir)

  # category id와 이름을 매핑하지만, 추후에 class 명만 활용
  def _categories(self):
    categories = {}
    for cls in self.labels['categories']:
        categories[cls['id']] = cls['name']
    return categories
  
  # annotation에서 모든 image의 파일명(절대 경로 아님)과 width, height 정보 저장
  def _load_images_info(self):
    images_info = {}
    for image in self.labels['images']:
        id = image['id']
        file_name = image['file_name']
        if file_name.find('\\') > -1:
            file_name = file_name[file_name.index('\\')+1:]
        w = image['width']
        h = image['height']
  
        images_info[id] = (file_name, w, h)

    return images_info

  # ms-coco의 bbox annotation은 yolo format으로 변환. 좌상단 x, y좌표, width, height 기반을 정규화된 center x,y 와 width, height로 변환
  def _bbox_2_yolo(self, bbox, img_w, img_h):
    # ms-coco는 좌상단 x, y좌표, width, height
    x, y, w, h = bbox[0], bbox[1], bbox[2], bbox[3]
    # center x좌표는 좌상단 x좌표에서 width의 절반을 더함. center y좌표는 좌상단 y좌표에서 height의 절반을 더함
    centerx = bbox[0] + w / 2
    centery = bbox[1] + h / 2
    # centerx, centery, width, height를 이미지의 width/height로 정규화
    dw = 1 / img_w
    dh = 1 / img_h
    centerx *= dw
    w *= dw
    centery *= dh
    h *= dh
    return centerx, centery, w, h
  
  # image와 annotation 정보를 기반으로 image명과 yolo annotation 정보 가공
  # 개별 image당 하나의 annotation 정보를 가지도록 변환
  def _convert_anno(self, images_info):
    anno_dict = dict()
    for anno in self.labels['annotations']:
      bbox = anno['bbox']
      image_id = anno['image_id']
      category_id = anno['category_id']

      image_info = images_info.get(image_id)
      image_name = image_info[0]
      img_w = image_info[1]
      img_h = image_info[2]
      yolo_box = self._bbox_2_yolo(bbox, img_w, img_h)

      anno_info = (image_name, category_id, yolo_box)
      anno_infos = anno_dict.get(image_id)
      if not anno_infos:
        anno_dict[image_id] = [anno_info]
      else:
        anno_infos.append(anno_info)
        anno_dict[image_id] = anno_infos
    return anno_dict

  # class 명을 파일로 저장하는 로직. 사용하지 않음
  def save_classes(self):
    sorted_classes = list(map(lambda x: x['name'], sorted(self.labels['categories'], key=lambda x: x['id'])))
    print('coco names', sorted_classes)
    with open('coco.names', 'w', encoding='utf-8') as f:
      for cls in sorted_classes:
          f.write(cls + '\n')
    f.close()
  # _convert_anno(images_info)로 만들어진 anno 정보를 개별 yolo anno txt 파일로 생성하는 로직
  # coco2yolo()에서 anno_dict = self._convert_anno(images_info)로 만들어진 anno_dict를 _save_txt()에 입력하여 파일 생성
  def _save_txt(self, anno_dict):
    # 개별 image별로 소스 image는 타겟이미지 디렉토리로 복사하고, 개별 annotation을 타겟 anno 디렉토리로 생성
    for k, v in anno_dict.items():
      # 소스와 타겟 파일의 절대 경로 생성
      src_img_filename = os.path.join(self.src_img_dir, v[0][0])
      tgt_anno_filename = os.path.join(self.tgt_anno_dir,v[0][0].split(".")[0] + ".txt")
      #print('source image filename:', src_img_filename, 'target anno filename:', tgt_anno_filename)
      # 이미지 파일의 경우 타겟 디렉토리로 단순 복사
      shutil.copy(src_img_filename, self.tgt_img_dir)
      # 타겟 annotation 출력 파일명으로 classid, bbox 좌표를 object 별로 생성
      with open(tgt_anno_filename, 'w', encoding='utf-8') as f:
        #print(k, v)
        # 여러개의 object 별로 classid와 bbox 좌표를 생성
        for obj in v:
          cat_name = self.coco_id_name_map.get(obj[1])
          # category_id는 class 명에 따라 0부터 순차적으로 부여
          category_id = self.coco_name_list.index(cat_name)
          #print('cat_name:', cat_name, 'category_id:', category_id)
          box = ['{:.6f}'.format(x) for x in obj[2]]
          box = ' '.join(box)
          line = str(category_id) + ' ' + box
          f.write(line + '\n')

  # ms-coco를 yolo format으로 변환
  def coco2yolo(self):
    print("loading image info...")
    images_info = self._load_images_info()
    print("loading done, total images", len(images_info))

    print("start converting...")
    anno_dict = self._convert_anno(images_info)
    print("converting done, total labels", len(anno_dict))

    print("saving txt file...")
    self._save_txt(anno_dict)
    print("saving done")

In [9]:
# 학습/검증/테스트용 images, labels 디렉토리 생성
!cd pig; mkdir images; mkdir labels;
!cd pig/images; mkdir train; mkdir val; mkdir test
!cd pig/labels; mkdir train; mkdir val; mkdir test

In [ ]:
# train 용 yolo 데이터 세트 생성

train_yolo_converter = COCO2YOLO(src_img_dir='/scratch/a1589a01/workspace/YOLO/data/Train.vol1/images_1', json_file='/scratch/a1589a01/workspace/YOLO/data/Train.vol1/json_1/seed221111/train.json',
                                 tgt_img_dir='/scratch/a1589a01/workspace/YOLO/pig/images/train', tgt_anno_dir='/scratch/a1589a01/workspace/YOLO/pig/labels/train')
train_yolo_converter.coco2yolo()

# val 용 yolo 데이터 세트 생성. 
val_yolo_converter = COCO2YOLO(src_img_dir='/scratch/a1589a01/workspace/YOLO/data/Train.vol1/images_1', json_file='/scratch/a1589a01/workspace/YOLO/data/Train.vol1/json_1/seed221111/val.json',
                                 tgt_img_dir='/scratch/a1589a01/workspace/YOLO/pig/images/val', tgt_anno_dir='/scratch/a1589a01/workspace/YOLO/pig/labels/val')
val_yolo_converter.coco2yolo()

# test 용 yolo 데이터 세트 생성. 
test_yolo_converter = COCO2YOLO(src_img_dir='/scratch/a1589a01/workspace/YOLO/data/Train.vol1/images_1', json_file='/scratch/a1589a01/workspace/YOLO/data/Train.vol1/json_1/seed221111/test.json',
                                 tgt_img_dir='/scratch/a1589a01/workspace/YOLO/pig/images/test', tgt_anno_dir='/scratch/a1589a01/workspace/YOLO/pig/labels/test')
test_yolo_converter.coco2yolo()

In [16]:
# 결과물 저장할 Directory 생성
!mkdir "/scratch/a1589a01/workspace/YOLO/ultra_workdir"

In [ ]:
import yaml
yaml.__version__

In [ ]:
# 아래 내용과 같이 데이터 위치와 class 내용을 담은 yaml파일을 만들어주어야 한다.

"""
train: /scratch/a1589a01/workspace/YOLO/pig/images/train/
val: /scratch/a1589a01/workspace/YOLO/pig/images/val/
test: /scratch/a1589a01/workspace/YOLO/pig/images/test/

#number of classes
nc: 1

#class names
names: ['pig']
"""

# 파일명은 "custom_data.yaml"으로 하면 된다

In [8]:
# cuda 사용 확인하기
import torch

cuda_id = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
# 배치 사이즈는 8, 에폭크기는 50으로 놓고 데이터 경로는 custom_data.yaml를 통해서 넣어준다.
!python train.py --img 640 --batch 8 --epochs 50 --data /scratch/a1589a01/workspace/YOLO/data/custom_data.yaml --weights yolov5l.pt \
                                     --project=/scratch/a1589a01/workspace/YOLO/ultra_workdir --name pig --exist-ok

In [ ]:
# 위에서 만든 모델로 test 셋에 적용해보기
!detect.py --source /scratch/a1589a01/workspace/YOLO/pig/images/test/ \
                            --weights /scratch/a1589a01/workspace/YOLO/ultra_workdir/pig/weights/best.pt --conf 0.2 \
                            --project=/scratch/a1589a01/workspace/YOLO/data/output --name=run_image --exist-ok --line-thickness 2